# Demand Curve Pipeline

This worksheet demonstrates a Wallaroo pipeline with data preprocessing, a model, and data postprocessing.

The model is a "demand curve" that predicts the expected number of units of a product that will be sold to a customer as a function of unit price and facts about the customer. Such models can be used for price optimization or sales volume forecasting.  This is purely a "toy" demonstration, but is useful for detailing the process of working with models and pipelines.

Data preprocessing is required to create the features used by the model. Simple postprocessing prevents nonsensical estimates (e.g. negative units sold).


## Connect to Wallaroo

The first task will be to import our libraries and connect to the Wallaroo we've installed in our cluster.  Note that our setup uses the `auth_type="user_password"` authentication setting, so modify this based on your own Wallaroo configuration.

In [1]:
import json
import wallaroo
import pandas
import numpy
import conversion
import os

In [2]:
os.environ["WALLAROO_SDK_CREDENTIALS"] = 'creds.json'
wl = wallaroo.Client(auth_type="user_password")

Now that the Wallaroo client has been initialized, we can create the workspace and call it `demandcurve-workspace`, then set it as our current workspace.

In [3]:
new_workspace = wl.create_workspace("demandcurve-workspace")
_ = wl.set_current_workspace(new_workspace)

Just to make sure, let's list our current workspace.  If everything is going right, it will show us we're in the `demandcurve-workspace`.

In [4]:
wl.get_current_workspace()

{'name': 'demandcurve-workspace', 'id': 2, 'archived': False, 'created_by': '7dbb3754-4c14-4730-8b77-33caeea7a2a0', 'created_at': '2022-03-28T16:28:21.625896+00:00', 'models': [], 'pipelines': []}

With our workspace established, we'll upload three models:

* `demand_curve_v1.onnx`: Our demand_curve model.  We'll store the upload configuration into `demand_curve_model`.
* `preprocess`:  Takes the data and prepares it for the demand curve model.  We'll store the upload configuration into `module_pre`.
* `postprocess`:  Takes the results from our demand curve model and prepares it for our display.  We'll store the upload configuration into `module_post`.

Note that the order we upload our models isn't important - we'll be establishing the actual process of moving data from one model to the next when we set up our pipeline.

In [5]:
# upload to wallaroo
demand_curve_model = wl.upload_model('demandcurve', "./demand_curve_v1.onnx").configure()

In [6]:
# load the preprocess module
module_pre = wl.upload_model("preprocess", "./preprocess.py").configure('python')

In [7]:
# load the postprocess module
module_post = wl.upload_model("postprocess", "./postprocess.py").configure('python')

With our models uploaded, we're going to create our own pipeline and give it three steps:

* First, start with the preprocess module we called `module_pre` to prepare the data.
* Second, we apply the data to our `demand_curve_model`.
* And finally, we prepare our data for output with the `module_post`.

In [8]:
# now make a pipeline
demandcurve_pipeline = (wl.build_pipeline("demand-curve-pipeline")
                        .add_model_step(module_pre)
                        .add_model_step(demand_curve_model)
                        .add_model_step(module_post))

And with that - let's deploy our model pipeline.  This usually takes about 45 seconds for the deployment to finish.

In [9]:
demandcurve_pipeline.deploy()

Waiting for deployment - this will take up to 45s ........ ok


{'name': 'demand-curve-pipeline', 'create_time': datetime.datetime(2022, 3, 28, 16, 28, 22, 313553, tzinfo=tzutc()), 'definition': "[{'ModelInference': {'models': [{'name': 'preprocess', 'version': 'b1f51290-ac47-4289-8a55-310507d52af5', 'sha': 'c328e2d5bf0adeb96f37687ab4da32cecf5f2cc789fa3a427ec0dbd2c3b8b663'}]}}, {'ModelInference': {'models': [{'name': 'demandcurve', 'version': '9cd1fcae-1fa1-4e12-8e67-d4a67f240a46', 'sha': '2820b42c9e778ae259918315f25afc8685ecab9967bad0a3d241e6191b414a0d'}]}}, {'ModelInference': {'models': [{'name': 'postprocess', 'version': '06e79dfe-623e-482e-95f6-bd6fa1b26264', 'sha': '4bd3109602e999a3a5013893cd2eff1a434fd9f06d6e3e681724232db6fdd40d'}]}}]"}

We can check the status of our pipeline to make sure everything was set up correctly:

In [10]:
demandcurve_pipeline.status()

{'status': 'Running',
 'details': None,
 'engines': [{'ip': '10.12.1.227',
   'name': 'engine-7cbf9b8d6d-xs64b',
   'status': 'Running',
   'reason': None,
   'pipeline_statuses': {'pipelines': [{'id': 'demand-curve-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'demandcurve',
      'version': '9cd1fcae-1fa1-4e12-8e67-d4a67f240a46',
      'sha': '2820b42c9e778ae259918315f25afc8685ecab9967bad0a3d241e6191b414a0d',
      'status': 'Running'},
     {'name': 'preprocess',
      'version': 'b1f51290-ac47-4289-8a55-310507d52af5',
      'sha': 'c328e2d5bf0adeb96f37687ab4da32cecf5f2cc789fa3a427ec0dbd2c3b8b663',
      'status': 'Running'},
     {'name': 'postprocess',
      'version': '06e79dfe-623e-482e-95f6-bd6fa1b26264',
      'sha': '4bd3109602e999a3a5013893cd2eff1a434fd9f06d6e3e681724232db6fdd40d',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.12.1.226',
   'name': 'engine-lb-85846c64f8-6l9rr',
   'status': 'Running',
   'reason': None}]}

Everything is ready.  Let's feed our pipeline some data.  We have some information prepared with the `daily_purchasses.csv` spreadsheet.  We'll start with just one row to make sure that everything is working correctly.

In [11]:
# read in some purchase data
purchases = pandas.read_csv('daily_purchases.csv')

# start with a one-row data frame for testing
subsamp_raw = purchases.iloc[0:1,: ]
subsamp_raw

# create the input dictionary from the original one-line dataframe
input_dict = conversion.pandas_to_dict(subsamp_raw)

result = demandcurve_pipeline.infer(input_dict)
result

Waiting for inference response - this will take up to 45s .. ok


[InferenceResult({'check_failures': [],
  'elapsed': 479657,
  'model_name': 'postprocess',
  'model_version': '06e79dfe-623e-482e-95f6-bd6fa1b26264',
  'original_data': {'colnames': ['Date',
                                 'cust_known',
                                 'StockCode',
                                 'UnitPrice',
                                 'UnitsSold'],
                    'query': [['2010-12-01', False, '21928', 4.21, 1]]},
  'outputs': [{'Json': {'data': [{'original': {'outputs': [{'Double': {'data': [6.68025518653071],
                                                                       'dim': [1,
                                                                               1],
                                                                       'v': 1}}]},
                                  'prediction': [6.68025518653071]}],
                        'dim': [1],
                        'v': 1}}],
  'pipeline_name': 'demand-curve-pipeline',
  'time': 1648484

We can see from the `prediction` field that the demand curve has a predicted slope of 6.68 from our sample data.  We can isolate that by specifying just the data output below.

In [12]:
result[0].data()

[array([6.68025519])]

# Bulk Inference

The initial test went perfectly.  Now let's throw some more data into our pipeline.  We'll draw 10 random rows from our spreadsheet, perform an inference from that, and then display the results and the logs showing the pipeline's actions.

In [14]:
# Let's do 10 rows at once (drawn randomly)
ix = numpy.random.choice(purchases.shape[0], size=10, replace=False)
output = demandcurve_pipeline.infer(conversion.pandas_to_dict(purchases.iloc[ix,: ]))

In [15]:
output[0].data()

[array([33.12532316,  6.77154593,  6.77154593, 40.57067889, 40.57067889,
         6.77154593, 33.12532316,  6.77154593,  9.11087115, 40.57067889])]

## Undeploy the Pipeline

Once we've finished with our demand curve demo, we'll undeploy the pipeline and give the resources back to our Kubernetes cluster.

In [ ]:
demandcurve_pipeline.undeploy()

Thank you for being a part of this demonstration.  If you have additional questions, please feel free to contact us at Wallaroo.